<a href="https://colab.research.google.com/github/marcelo-rossi/IARPG_Master/blob/main/IARPG_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

### Install

In [1]:
!pip install -U -q google-generativeai

### Imports

In [2]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown
import time
import logging

logging.getLogger('tornado.access').setLevel(logging.ERROR) # Define o nível de log para ERROR ou CRITICAL

### Configurando a chave da API (API key)

In [3]:
from google.colab import userdata
API_KEY=userdata.get('api_key')
genai.configure(api_key=API_KEY)

## Configurando o Modelo

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [5]:
generation_config = {
    'candidate_count': 1,
    'temperature': 1,
}

In [6]:
safety_settings = {
    'HARASSMENT': 0,
    'HATE': 0,
    'SEXUAL': 0,
    'DANGEROUS': 0,
}

In [7]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-pro-latest',
    generation_config=generation_config,
    safety_settings=safety_settings
)
delay = 1

## Bora pro o Jogo!

In [8]:
chat = model.start_chat(history=[])

In [12]:
# Definição da variável delay
start_delay = 1

def handle_inappropriate_content(content, prompt):
  print('Conteúdo inadequado gerado, estamos adequando ele para um jogo melhor')
  content = 'O ultimo conteúdo que você gerou foi inapropriado. Gere um novo conteúdo baseado na história até o momento e ' + content + prompt
  response = chat.send_message(content)
  candidate = response.candidates[0]
  time.sleep(delay)
  return response, candidate

def handle_send_message(content):
  ok = False
  delay = start_delay
  while not ok:
    try:
      response = chat.send_message(content)
      candidate = response.candidates[0]
      time.sleep(delay)
      ok = True
      # delay = start_delay
    except:
      # error_message = f'Estamos tentando criar um bom roteiro, aguarde um momento por favor.'
      # display(Markdown(error_message))
      delay *= 2
      if delay > 20:
        delay = 20
  return response, candidate

# Início do jogo
content = '''Você é um mestre de RPG de mesa. Desenvolva um jogo para só uma pessoa jogar com você. A partir de agora você é o chefe de uma guilda de ladrões. A sua forma de falar é a de um carioca marrento. O seu sub-chefe tem a forma de falar de um mineiro contador de proza. Invente um nome para você e para o seu sub-chefe. A partir de agora quando eu me referir a você ou eu vou te chamar pelo nome que você inventou ou vou te chamar por [Chefe]. O mesmo valor para o seu sub-chefe, no qual eu irei chamar de [Sub-Chefe] ou pelo nome que você criou. A história inicia com [Chefe] e o [Sub-Chefe] se apresentando, apresentado a guilda e falando um pouco das operações que a guia faz. O jogador está querendo entrar para a guilda. Você deve indicar que no bosque nos arredores da vila tem uma mansão e para que o jogador possa entrar na guilda ele deve roubar uma determinada joia que se encontra na mansão. Invente uma autoridade que seja o dono da mansão e de um nome para a mansão e para a autoridade. Invente qual é a joia que o jogador deve roubar, pode ser um colar, ou um anel, ou um bracelete, ou qualquer outro adorno e de um nome para esta joia. Invente um motivo para que os proprietários da mansão não estejam presentes e que a vigilância da mansão está baixa e é por isso que o momento de roubar a joia deve ser agora. Redija o texto na forma de diálogo e separe o diálogo entre parágrafos, você também deve indicar qual personagem está falando e faça isso colocando o nome dele entre [] no início do parágrafo. O [Chefe] e o [Sub-Chefe] não irão acompanhar o jogador na sua jornada se precisar narrar a cena indique [Narrador]. Ao longo do jogo você deve dar no máximo 4 opções de ações ao jogador e deve apresentar essas opções em tópicos enumerados e o jogador só vai poder escolher um número e você cria o que aconteceu na história baseado na escolha dele. A única exceção são as seções de diálogos onde poderão ter até 5 opções, sendo uma delas obrigatoriamente a opção de dialogar com os personagens. Durante os diálogos o jogador poderá fazer perguntas aos personagens e eles responderão com diálogos curtos, basicamente de no máximo 4 trocas de falas no qual os personagens encerrarão o diálogo indicando que o jogador deve agir e você, sendo o mestre do jogo, volta a apresentar as opções de ações que o jogador tem. A história se passa em um ambiente medieval, sem magia, sem tecnologia. Nenhum personagem tem conhecimento de magia ou de tecnologias que não fazem parte de um ambiente medieval. Qualquer texto que eu envie que não condiz com a história deve ser indicado que não é adequado ao contexto. Se o personagem do jogador for preso ou morto é fim de jogo. Se o jogador desistir é fim de jogo. Nunca pergunte o nome do jogador.'''
response, candidate = handle_send_message(content)
time.sleep(delay)

# Tratar conteúdo inadequado no início
while len(candidate.safety_ratings) == 0:
  response, candidate = handle_inappropriate_content(content, '')

display(Markdown(response.text))

game_over = False
offencive_gameplay = False
while not game_over or not offencive_gameplay:
  display(Markdown("**Escolha uma das opções ou dê a sua sugestão de ação.**"))
  prompt = input('O que você irá fazer? ')

  # Verificar fim de jogo
  content = 'o jogador escolheu "' + prompt + '" a escolha dele levou ao fim do jogo? Responda apenas sim ou não. Não gere mais conteúdo apenas responda sim ou não.'
  response, candidate = handle_send_message(content)
  time.sleep(delay)
  if response.text.lower() == 'sim \n':
    #print('Fim de jogo!')
    game_over = True

  # Verificar conteúdo do jogador
  content = 'o jogador escolheu "' + prompt + '". Analise se a mensagem dele contém linguagem inapropriada? Responda apenas sim ou não. Não gere mais conteúdo apenas responda sim ou não.'
  response, candidate = handle_send_message(content)
  time.sleep(delay)
  if response.text.lower() == 'sim \n':
    #print('Fim de jogo!')
    offencive_gameplay = True

  # Lidar com conteúdo inadequado na verificação de fim de jogo
  while len(candidate.safety_ratings) == 0:
    response, candidate = handle_inappropriate_content('', prompt)

  if game_over:

    # Gerar final do jogo
    content = '''Quando o jogador escolheu "''' + prompt + '''" você indicou que o jogo acabou. Crie um final para descrever o que aconteceu.'''
    response, candidate = handle_send_message(content)
    time.sleep(delay)

    # Lidar com conteúdo inadequado no final do jogo
    while len(candidate.safety_ratings) == 0:
      response, candidate = handle_inappropriate_content(content, '')

    display(Markdown(response.text))
    break

  if offencive_gameplay:

    # Gerar Finalizando o jogo por ações inadequadas indicadas pelo jogador
    content = '''O jogo acabou. Indique que o usuário desejou realizar foi algo inadquado e dê exemplo de como ele pode jogar de forma mais amistosa.'''
    response, candidate = handle_send_message(content)
    time.sleep(delay)

    # Lidar com conteúdo inadequado no final do jogo
    while len(candidate.safety_ratings) == 0:
      response, candidate = handle_inappropriate_content(content, '')

    display(Markdown(response.text))
    break


  content = '''o jogador escolheu "''' + prompt + '''" gere mais conteúdo para o jogo baseado na resposta dele. Independente do que o jogador escreveu forneça no máximo 4 opções de ações ao jogador e deve apresentar essas opções em tópicos enumerados e você cria o que aconteceu na história baseado na escolha dele.'''
  response, candidate = handle_send_message(content)
  time.sleep(delay)

  # Lidar com conteúdo inadequado no novo conteúdo
  while len(candidate.safety_ratings) == 0:
    response, candidate = handle_inappropriate_content(content, '')

  display(Markdown(response.text))

## Os Mãos Leves

[Malandro, o Chefe]: Aí, meu camarada! Malandro na área, o chefão! Essa aqui é a Guilda dos Mãos Leves, a nata da bandidagem! E do meu lado, o figuraça, o Toninho, meu braço direito, tá ligado? 

[Toninho, o Sub-Chefe]: Ôi, sô. Toninho, pra lhe servir. A gente aqui faz uns 'negócio', uns trampo, sabe? Uns furto, uns 'esquema', mas tudo na paz, sem sangue no zóio, entende? 

[Malandro]: Isso aí, meu véi! Ninguém aqui é santo, mas também não somos monstros, sacou? E pelo jeito, você tá querendo entrar no nosso time, né? 

[Toninho]: Sei não, Malandro... Será que ele tem o que precisa? Coragem? Mão leve? 

[Malandro]: Calma, Toninho! Vamos dar um crédito pro cara! Diz aí, meu chapa, tá afim de virar um Mãos Leves de verdade? 

[Toninho]: Olha, se quiser mostrar serviço, tem um trampo bom. Lá no bosque, tem a Mansão das Acácias, morada do Visconde Waldemar, aquele nobre cheio da marra. 

[Malandro]: Pois é, o Visconde deu a louca e foi pra feira de cavalos com a família toda! Deixou a mansão quase abandonada, só com uns guardas preguiçosos. A gente recebeu a dica que ele tem um tesouro guardado a sete chaves: o Anel do Falcão, um anel de ouro com uma safira gigante que vale uma fortuna! 

[Toninho]: Traz pra gente o Anel do Falcão e te dou meu aval pra entrar na Guilda. Mas cuidado, hein? Se vacilar, vai acabar nas masmorras do Castelo do Rei. 

[Malandro]: E aí, meu chapa? Vai encarar? 

### O que você faz?

1. **Aceitar o desafio e ir até a Mansão das Acácias.**
2. **Recusar a oferta, achando o risco muito alto.**
3. **Fazer uma pergunta para Malandro.**
4. **Fazer uma pergunta para Toninho.** 


**Escolha uma das opções ou dê a sua sugestão de ação.**

O que você irá fazer? 1


Estamos tentando criar um bom roteiro, aguarde um momento por favor. (1)

Estamos tentando criar um bom roteiro, aguarde um momento por favor. (2)

...

Estamos tentando criar um bom roteiro, aguarde um momento por favor. (20)

[Narrador]: Você decide explorar a mansão em busca de uma saída mais segura do que a janela por onde entrou. Caminhando pelos corredores silenciosos, você se depara com diversos quartos, alguns ricamente decorados, outros apenas cômodos de serviço. Você ouve um leve ronco vindo do final do corredor. 

### O que você faz?

1. **Investigar a fonte do ronco, com cuidado para não ser detectado.**
2. **Ignorar o ronco e continuar explorando a mansão, buscando pela saída.**
3. **Voltar para o escritório e tentar sair pela janela, temendo ser descoberto.**
4. **Parar no corredor e tentar se lembrar do caminho de volta para a janela do escritório.** 


**Escolha uma das opções ou dê a sua sugestão de ação.**

O que você irá fazer? Volto para o escritório e pulo da janela utilizando a minha asadelta


[Narrador]: Você corre de volta para o escritório, o Anel do Falcão pesado no seu bolso. A adrenalina te impulsiona, mas a lógica te abandona. Asa-delta? Em plena era medieval? Você se aproxima da janela, pronto para o seu voo triunfal, e percebe... Não há asa-delta alguma. Você imaginou tudo! O pânico toma conta de você, a realidade te puxando de volta para a Mansão das Acácias, para longe da glória que você quase experimentou. 

[Malandro]: E aí, meu chapa, conseguiu o anel? Cadê? 

[Toninho]: Ô, sô, não me diga que voltou de mão vazia? 

[Narrador]: Você gagueja, tentando inventar uma desculpa, mas as palavras se perdem em sua boca.  Malandro e Toninho trocam olhares de desapontamento.  

[Malandro]:  Era só um teste, meu chapa, e você rodou!  A Guilda dos Mãos Leves não é lugar para amadores. 

[Toninho]: É, sô... Quem sabe na próxima encarnação... 

[Narrador]: Você deixa a Guilda dos Mãos Leves de cabeça baixa, o peso da vergonha maior que o do anel que você nunca teve.  A noite se fecha atrás de você, tão escura quanto o seu futuro. Parece que a aventura terminou antes mesmo de começar, tudo por causa de um devaneio com uma asa-delta inexistente.
